In [ ]:
#期权数据
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 读入期权价格行情数据
option_price = pd.read_csv('data/option.csv')
option_price

# 从存续时间比较长的合约中选择一个认购期权
contracts = option_price.groupby('ticker').size().sort_values(ascending=False) #对合约groupby后返回每组的数量并降序排序，选数量较多的

# 选择卖出一个深度实值期权对冲风险(存续时间较长就可以了，也可以选虚值和平值期权)
ticker = '510050C1512M03000'

# 获取期权有效期
option_price[option_price.ticker == ticker][ 'tradeDate']

# 提取特定期权合约的行情数据
data_option = option_price[option_price.ticker == ticker]
data_option.set_index('tradeDate', inplace=True)

# 获取行情开始和结束时间
start, end = data_option.index.values[[0,-2]]   # 此处有效期少选一天是为了避免计算 delta 时出现除数为0的情况
print(f'start:\t{start} \nend:\t{end}')

#标的资产数据
from math import log, e, sqrt, pi
import tushare as ts

# 获取50ETF数据
asset_data = ts.get_k_data('510050', '2015-01-01')

# 将日期列设为索引
asset_data.set_index('date', inplace=True)

# 计算标的资产的波动率
asset_data['d_return'] = asset_data.close.pct_change()
asset_data['volatility'] = asset_data.d_return.rolling(60).std() * sqrt(252) #年化历史波动率
asset_data = asset_data[(asset_data.index <= end) & (asset_data.index >= start)] #取与期权数据相同的时间段

# 合并期权数据和基础资产数据
portfolio = pd.merge(asset_data, data_option, how='left', left_index=True, right_index=True)
portfolio = portfolio[['close', 'volatility', 'closePrice']]

##静态Delta对冲
from datetime import datetime
from scipy import stats

# 计算期权delta
def get_delta(S, X, r, q, sigma, T, opt_type='call'):
    """
    根据BSM模型计算期权价格
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年为单位的期权到期时间
    param opt_type: 'call'或'put'
    return 返回期权的delta值
    """

    d1 = (log(S / X) + (r - q + sigma ** 2 / 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    N1 = stats.norm.cdf(d1)
    delta_c = e ** (-q * T) * N1
    if opt_type.lower()[0] == 'c':
        return delta_c
    else:
        return delta_c - 1

def get_timedelta(start: str, end: str): #计算两个日期间间隔的天数
    """
    定义函数计算两个日期之间的日历天数
    param start： 起始日期，字符串格式
    param end： 终值日期，字符串格式
    return：返回两个日期之间的日历天数
    """
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    timedelta = end - start
    return timedelta.days

timedelta = get_timedelta(start, end)
timedelta

get_delta(S=3.2, X=3, r=0.0246, q=0, sigma=0.27, T=timedelta/365, opt_type='c') #注意timedelta是工作日则除252，是工作日+周末则除365

# 计算原始头寸的 delta
option_factor = 10000           # 50etf期权合约对应10000份50etf基金份额
n_options = -100                # 做空期权合约时，数量为负

opt_delta = get_delta(S=3.2, X=3, r=0.0246, q=0, sigma=0.27, T=timedelta/365, opt_type='c')
portfolio_delta = opt_delta * option_factor * n_options #做空100张期权合约*每个合约对应10000个期权*delta得到这100张期权合约需要多少单位标的进行对冲
portfolio_delta                 # -100张认购期权的delta值

# 计算对冲头寸的数量
hedge_delta = 1 #由于使用标的对冲期权，所以标的的hedge_delta为1，如果使用非标的(比如期权)对冲期权，则hedge_delta不为1
n_asset = -round(portfolio_delta / hedge_delta / 1000, 0) * 1000  # 由于50etf基金(标的)的交易份额需要为1000的正数倍，所以除hedge_delta和1000取整得到所需合约数再乘1000得到需要多少单位标的进行对冲
n_asset

def calc_hedge(n_options, opt_delta, hedge_delta=1): #将静态对冲写成一个函数
    """
    计算delta对冲的标的资产数量
    param n_options: 需要对冲的期权数量，正值代表做多，负值代表做空
    param opt_delta：期权的delta(乘以期权乘数以后的结果，50etf期权的期权乘数是10000）
    param hedge_delta：用于对冲的资产的delta，对于股票来说为1
    return： 返回维持delta中性所需要的标的资产的数量，保留整数
    """
    portfolio_delta = n_options * opt_delta
    n_asset = -round(portfolio_delta / hedge_delta / 1000, 0) * 1000  # 50etf基金的交易份额需要为1000的整数倍
    return n_asset

calc_hedge(-100, 6834.77, 1)

##动态Delta对冲
# 初始参数
n_call = -100

X = 3
r = 0.0246
q = 0
opt_type = 'c'

def apply_func(s, n_option, X=3, r=0.0246, q=0, end_date='2017-06-28', opt_type='c'): #应用于DataFrame的每一行，用于计算用于对冲的现货头寸
    """
    定义apply的回调函数，作用是处理apply函数传入的DataFrame中的一行数据
    param s: DataFrame中的一行数据
    param n_option: 期权合约的数量
    param X: 期权的行权价格
    param r: 无风险收益率
    param q: 基础资产的股息率
    param end_date：期权的到期时间
    param opt_type: 期权的类型，call/put
    """
    S = s['close']
    start_date = s.name           # apply传进函数的数据有.name方法，用于返回该行/列数据的index/列标签
    sigma = s['volatility']
    opt_price = s['closePrice']
    
    timedelta = get_timedelta(start_date, end_date)
    T = timedelta / 365
    option_delta = 10000 * get_delta(S, X, r, q, sigma, T, opt_type)
    return calc_hedge(n_option, option_delta) #用之前定义的calc_hedge计算对冲头寸

# 计算每日对冲账户的现货持仓
portfolio['hedge_position'] = portfolio.apply(apply_func, axis=1, n_option=-100, X=3, r=0.0246, q=0, end_date='2015-12-23', opt_type='c')
# 计算对冲账户现货的市场价值
portfolio['hedge_value'] = portfolio.hedge_position * portfolio.close
# 计算每日需要调整的现货资产数量，正数代表买入基础资产，负数代表卖出基础资产
portfolio['hedge_position_change'] = portfolio.hedge_position.diff()

# 计算调仓产生的现金流，买入基础资产现金流为负，卖出现金流为正
portfolio['hedge_cash'] = - portfolio.close * portfolio.hedge_position_change
# 计算对冲账户整体的价值
portfolio['hedge_total'] = portfolio.hedge_value + portfolio.hedge_cash 
# 计算对冲账户整体的价值变动
portfolio['hedge_total_change'] = portfolio.hedge_total.diff()

# 计算持仓期权的市场价值
portfolio['option_value'] = - portfolio.closePrice * 10000 * 100
# 计算持仓期权的市场价格变动
portfolio['option_value_change'] = portfolio.option_value.diff()

# 计算组合(期权+标的)的价值变动
portfolio['portfolio_change'] = portfolio[['hedge_total_change', 'option_value_change']].sum(axis=1)
# 计算初始权利金
option_premium = portfolio.closePrice.iat[0] * 10000 * 100
# 计算组合的价值
portfolio['portfolio_value'] = portfolio.portfolio_change + option_premium
# 使用权利金对组合价值标准化
portfolio['standardized_portofolio_value'] = portfolio.portfolio_value / option_premium

# portfolio.to_csv('动态delta对冲.csv')
portfolio

portfolio['standardized_portofolio_value'].plot(figsize=(12, 6)) #delta对冲只有在标的价格变动较小时才能保证整体对冲组合价值接近不变，如果比较大，delta本身的值变动较大，改善方法是提高对冲频率(相当于每个时间间隔标的价格的变动幅度减小，但会增加对冲成本)、使用gamma对冲。图形开始波动很大后来波动较小是因为随着到期日临近，delta逐渐趋近于0，期权以不行权状态过期。
plt.legend()
plt.show()